In [1]:
import duckdb
from deltalake import DeltaTable
import os
from datetime import datetime
def OneLake(Workspace_Path) :
  now=int(datetime.now().timestamp())
  List_tables = [ f.name for f in os.scandir(Workspace_Path) if f.is_dir() ]
  for tbl in List_tables :
    dt = DeltaTable(Workspace_Path+ tbl)
    delta = [Workspace_Path + tbl + "/" + i for i in dt.files()]
    duckdb.sql(' set enable_progress_bar = False ;install spatial; load spatial ')
    duckdb.sql(f' select * from parquet_metadata({delta})').to_view(tbl+'_stat')
    duckdb.sql(f''' COPY {tbl}_stat TO '{now}_{tbl}.xlsx' WITH (FORMAT GDAL, DRIVER 'xlsx')''')
    print(tbl)
    duckdb.sql(f''' select split_part(file_name, '/', 9) as file ,row_group_id , max(row_group_num_rows) as Nbr_rows from  {tbl}_stat group by all ''').show()
    duckdb.sql(f' select * from parquet_scan({delta})').to_view(tbl)

In [3]:
Workspace_Path1 = 'C:/Users/mimoune.djouallah/OneLake - Microsoft/fabricworkspacetest/TPCH.Lakehouse/Tables/'
OneLake(Workspace_Path1)

customer
┌─────────────────────────────────────────────────────────────────────┬──────────────┬──────────┐
│                                file                                 │ row_group_id │ Nbr_rows │
│                               varchar                               │    int64     │  int64   │
├─────────────────────────────────────────────────────────────────────┼──────────────┼──────────┤
│ part-00000-62f24620-529b-44c4-b1d5-be64557aaaa7-c000.snappy.parquet │            0 │   810000 │
│ part-00000-62f24620-529b-44c4-b1d5-be64557aaaa7-c000.snappy.parquet │            1 │   690000 │
└─────────────────────────────────────────────────────────────────────┴──────────────┴──────────┘

customerrrrr
┌─────────────────────────────────────────────────────────────────────┬──────────────┬──────────┐
│                                file                                 │ row_group_id │ Nbr_rows │
│                               varchar                               │    int64     │  int64  

In [ ]:
duckdb.sql(' describe')